In [1]:
!pip install  LughaatNLP

In [8]:
# Load the dataset
data = pd.read_excel('cleaned_parallel-corpus.xlsx')
# Filter out rows with less than 3 words in either English or Urdu
data = data[data['SENTENCES'].str.split().str.len() >= 3]
data = data[data['MEANING'].str.split().str.len() >= 3]
data.shape

(22438, 2)

# Data Preprocessing

In [39]:

# Ensure reproducibility
torch.manual_seed(0)
random.seed(0)
np.random.seed(0)



# Load the dataset
data = pd.read_excel('cleaned_parallel-corpus.xlsx')
# Filter out rows with less than 3 words in either English or Urdu
data = data[data['SENTENCES'].str.split().str.len() >= 3]
data = data[data['MEANING'].str.split().str.len() >= 3]

data = data[data['SENTENCES'].str.split().str.len() <= 20]
data = data[data['MEANING'].str.split().str.len() <= 20]
# Data Preprocessing
def normalize_string(s):
    """Lowercase, trim, and remove non-letter characters."""
    s = s.lower().strip()
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

def tokenize(sentence):
    """Tokenize a sentence into words."""
    return sentence.strip().split()

# Preprocess English sentences
data['SENTENCES'] = data['SENTENCES'].apply(normalize_string)
data['English_tokens'] = data['SENTENCES'].apply(tokenize)

   return tokens

data['Urdu_tokens'] = data['MEANING'].apply(urdu_text_processing.urdu_tokenize)

# Build Vocabulary
def build_vocab(sentences, min_freq=1):
    """Build vocabulary with word frequencies above min_freq."""
    word_counts = Counter()
    for sentence in sentences:
        word_counts.update(sentence)
    vocab = ['<pad>', '<sos>', '<eos>', '<unk>']  # Special tokens
    vocab += [word for word, count in word_counts.items() if count >= min_freq]
    word2idx = {word: idx for idx, word in enumerate(vocab)}
    idx2word = {idx: word for word, idx in word2idx.items()}
    return word2idx, idx2word

eng_word2idx, eng_idx2word = build_vocab(data['English_tokens'], min_freq=1)
urd_word2idx, urd_idx2word = build_vocab(data['Urdu_tokens'], min_freq=1)

eng_vocab_size = len(eng_word2idx)
urd_vocab_size = len(urd_word2idx)

print("English Vocabulary Size:", eng_vocab_size)
print("Urdu Vocabulary Size:", urd_vocab_size)

# Convert sentences to indices
def sentence_to_indices(sentence, word2idx):
    """Convert a tokenized sentence to indices, adding <sos> and <eos> tokens."""
    return [word2idx.get('<sos>')] + \
           [word2idx.get(word, word2idx.get('<unk>')) for word in sentence] + \
           [word2idx.get('<eos>')]

data['English_indices'] = data['English_tokens'].apply(lambda x: sentence_to_indices(x, eng_word2idx))
data['Urdu_indices'] = data['Urdu_tokens'].apply(lambda x: sentence_to_indices(x, urd_word2idx))

# Split the dataset
train_data, test_data = train_test_split(data, test_size=0.1, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.1, random_state=42)

print(f"Training samples: {len(train_data)}")
print(f"Validation samples: {len(val_data)}")
print(f"Test samples: {len(test_data)}")

# Create Dataset and DataLoader
class TranslationDataset(Dataset):
    def __init__(self, data):
        self.data = data.reset_index(drop=True)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        eng_seq = self.data.loc[idx, 'English_indices']
        urd_seq = self.data.loc[idx, 'Urdu_indices']
        return torch.tensor(eng_seq, dtype=torch.long), torch.tensor(urd_seq, dtype=torch.long)

def collate_fn(batch):
    """Collate function to pad sequences to the same length."""
    eng_seqs, urd_seqs = zip(*batch)
    eng_lens = [len(seq) for seq in eng_seqs]
    urd_lens = [len(seq) for seq in urd_seqs]
    eng_seqs_padded = nn.utils.rnn.pad_sequence(eng_seqs, padding_value=eng_word2idx['<pad>'], batch_first=True)
    urd_seqs_padded = nn.utils.rnn.pad_sequence(urd_seqs, padding_value=urd_word2idx['<pad>'], batch_first=True)
    return eng_seqs_padded, urd_seqs_padded, eng_lens, urd_lens

batch_size = 16

train_dataset = TranslationDataset(train_data)
val_dataset = TranslationDataset(val_data)
test_dataset = TranslationDataset(test_data)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, collate_fn=collate_fn)


<ipython-input-39-4fde82b1486b>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['SENTENCES'] = data['SENTENCES'].apply(normalize_string)
<ipython-input-39-4fde82b1486b>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['English_tokens'] = data['SENTENCES'].apply(tokenize)


English Vocabulary Size: 8351
Urdu Vocabulary Size: 8469
Training samples: 13147
Validation samples: 1461
Test samples: 1624


In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import random
import re
import numpy as np
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from collections import Counter
import nltk
from nltk.translate.bleu_score import corpus_bleu
from LughaatNLP import LughaatNLP
urdu_text_processing = LughaatNLP()

import torch.nn.functional as F
import matplotlib.pyplot as plt


# RNN Model

In [18]:
c

In [16]:
# Model Architecture with Attention using RNN
class Seq2SeqRNN_Attention(nn.Module):
    def __init__(self, encoder, decoder, device='cuda'):
        super(Seq2SeqRNN_Attention, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, src_lengths, trg_lengths, teacher_forcing_ratio=0.5):
        batch_size = src.size(0)
        max_trg_len = trg.size(1)
        trg_vocab_size = self.decoder.fc.out_features

        outputs = torch.zeros(batch_size, max_trg_len, trg_vocab_size).to(self.device)

        encoder_outputs, hidden = self.encoder(src, src_lengths)

        input = trg[:, 0]  # <sos>

        for t in range(1, max_trg_len):
            output, hidden = self.decoder(input.unsqueeze(1), hidden, encoder_outputs)
            outputs[:, t] = output
            top1 = output.argmax(1)
            teacher_force = random.random() < teacher_forcing_ratio
            input = trg[:, t] if teacher_force else top1
        return outputs

# Instantiate the models with hyperparameter tuning
embedding_dim = 512
hidden_size = 1024
num_layers = 1
learning_rate = 0.0005
num_epochs = 3

# CUDA configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Instantiate the models with hyperparameter tuning
encoder = EncoderRNN(eng_vocab_size, hidden_size, embedding_dim, num_layers).to(device)
decoder = DecoderRNN_Attention(hidden_size, urd_vocab_size, embedding_dim, num_layers).to(device)
model = Seq2SeqRNN_Attention(encoder, decoder, device=device).to(device)

# Training Setup
criterion = nn.CrossEntropyLoss(ignore_index=urd_word2idx['<pad>'])
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [17]:





# Modified training loop with model saving after each epoch
def train_model(model, iterator, optimizer, criterion, clip=1, teacher_forcing_ratio=0.5):
    model.train()
    epoch_loss = 0
    for i, (src, trg, src_lengths, trg_lengths) in enumerate(iterator):
        src, trg = src.to(device), trg.to(device)
        optimizer.zero_grad()
        output = model(src, trg, src_lengths, trg_lengths, teacher_forcing_ratio)
        output_dim = output.shape[-1]
        output = output[:, 1:].reshape(-1, output_dim)
        trg = trg[:, 1:].reshape(-1)
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(iterator)

def evaluate_model(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, (src, trg, src_lengths, trg_lengths) in enumerate(iterator):
            src, trg = src.to(device), trg.to(device)
            output = model(src, trg, src_lengths, trg_lengths, teacher_forcing_ratio=0)
            output_dim = output.shape[-1]
            output = output[:, 1:].reshape(-1, output_dim)
            trg = trg[:, 1:].reshape(-1)
            loss = criterion(output, trg)
            epoch_loss += loss.item()
    return epoch_loss / len(iterator)

# Training loop with saving model at each epoch
best_valid_loss = float('inf')
train_losses = []
valid_losses = []

for epoch in range(num_epochs):
    tf_ratio = max(0.5 - epoch * 0.02, 0)  # Gradually decrease teacher forcing ratio
    train_loss = train_model(model, train_loader, optimizer, criterion, teacher_forcing_ratio=tf_ratio)
    valid_loss = evaluate_model(model, val_loader, criterion)
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)

    # Save model after each epoch
    torch.save(model.state_dict(), f'seq2seq-rnn-attention-epoch{epoch+1}.pt')

    # Save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'seq2seq-rnn-attention-best.pt')

    print(f'Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.3f} | Val Loss: {valid_loss:.3f}')




# Load the best model
# Load the best model for inference
model.load_state_dict(torch.load('seq2seq-rnn-attention-best.pt'))

# Evaluation
def translate_sentence(model, sentence, eng_word2idx, urd_idx2word, max_length=50):
    model.eval()
    tokens = tokenize(normalize_string(sentence))
    indices = sentence_to_indices(tokens, eng_word2idx)
    # Move src_tensor to the same device as the model
    src_tensor = torch.tensor(indices, dtype=torch.long).unsqueeze(0).to(device)
    src_length = [len(indices)]
    with torch.no_grad():
        encoder_outputs, hidden = model.encoder(src_tensor, src_length)
    trg_indices = [urd_word2idx['<sos>']]
    for i in range(max_length):
        # Move trg_tensor to the same device as the model
        trg_tensor = torch.tensor([trg_indices[-1]], dtype=torch.long).to(device)
        with torch.no_grad():
            output, hidden = model.decoder(trg_tensor.unsqueeze(1), hidden, encoder_outputs)
            pred_token = output.argmax(1).item()
            trg_indices.append(pred_token)
            if pred_token == urd_word2idx['<eos>']:
                break
    trg_tokens = [urd_idx2word.get(idx, '<unk>') for idx in trg_indices]
    return ' '.join(trg_tokens[1:-1])  # Exclude <sos> and <eos>

def calculate_bleu(model, data, eng_word2idx, urd_idx2word):
    references = []
    hypotheses = []
    for idx in range(len(data)):
        src_tokens = data.iloc[idx]['English_tokens']
        trg_tokens = data.iloc[idx]['Urdu_tokens']
        src_sentence = ' '.join(src_tokens)
        reference = [' '.join(trg_tokens)]
        hypothesis = translate_sentence(model, src_sentence, eng_word2idx, urd_idx2word)
        references.append([reference[0].split()])
        hypotheses.append(hypothesis.split())
    bleu_score = corpus_bleu(references, hypotheses)
    return bleu_score

bleu_score = calculate_bleu(model, test_data, eng_word2idx, urd_idx2word)
print(f'Test BLEU Score (RNN with Attention): {bleu_score*100:.2f}')

# Example Translations
test_sentences = [
    "How can I communicate with my parents?",
    "If you’ve asked yourself such questions, you’re not alone.",
    "Thumbs up",
    "Food was good but service was very slow."
]

for sentence in test_sentences:
    translation = translate_sentence(model, sentence, eng_word2idx, urd_idx2word)
    print(f'English: {sentence}')
    print(f'Urdu Translation (RNN with Attention): {translation}')
    print()


Using device: cuda
Epoch 1/3 | Train Loss: 5.382 | Val Loss: 5.692
Epoch 2/3 | Train Loss: 4.869 | Val Loss: 5.640
Epoch 3/3 | Train Loss: 4.551 | Val Loss: 5.557


<ipython-input-17-6e5bdd3ee6c8>:169: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('seq2seq-rnn-attention-best.pt'))


Test BLEU Score (RNN with Attention): 6.30
English: How can I communicate with my parents?
Urdu Translation (RNN with Attention): کیا اپنے والدین کو اپنے والدین کے ساتھ مارسیلز

English: If you’ve asked yourself such questions, you’re not alone.
Urdu Translation (RNN with Attention): تم تم آپ کو دوبارہ سامنا سامنا سامنا سامنا سامنا سامنا سامنا سامنا سامنا سامنا سامنا سامنا سامنا سامنا سامنا سامنا سامنا سامنا سامنا سامنا سامنا سامنا سامنا سامنا سامنا سامنا سامنا سامنا سامنا سامنا سامنا سامنا سامنا سامنا سامنا سامنا سامنا سامنا سامنا سامنا سامنا سامنا سامنا سامنا

English: Thumbs up
Urdu Translation (RNN with Attention): بدترین ایلن کو بہتر

English: Food was good but service was very slow.
Urdu Translation (RNN with Attention): کھانا اچھا تھا لیکن سروس تھوڑی سست تھی ۔



# LSTM Implementation

In [40]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import random
import re
import numpy as np
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from collections import Counter
import nltk
from nltk.translate.bleu_score import corpus_bleu

import torch.nn.functional as F
import matplotlib.pyplot as plt

# Model Architecture with Attention using LSTM
class EncoderLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, embedding_dim, num_layers=1):
        super(EncoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers=num_layers, batch_first=True)

    def forward(self, x, x_lengths):
        # x: (batch_size, seq_length)
        embedded = self.embedding(x)  # (batch_size, seq_length, embedding_dim)
        packed = nn.utils.rnn.pack_padded_sequence(embedded, x_lengths, batch_first=True, enforce_sorted=False)
        outputs, (hidden, cell) = self.lstm(packed)
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs, batch_first=True)
        return outputs, (hidden, cell)  # outputs: (batch_size, seq_length, hidden_size)

class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.attn = nn.Linear(hidden_size * 2, hidden_size)
        self.v = nn.Parameter(torch.rand(hidden_size))

    def forward(self, hidden, encoder_outputs):
        # hidden: (batch_size, hidden_size)
        # encoder_outputs: (batch_size, seq_len, hidden_size)
        timestep = encoder_outputs.size(1)
        hidden = hidden.unsqueeze(1).repeat(1, timestep, 1)  # (batch_size, seq_len, hidden_size)
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))  # (batch_size, seq_len, hidden_size)
        energy = energy.transpose(1, 2)  # (batch_size, hidden_size, seq_len)
        v = self.v.repeat(encoder_outputs.size(0), 1).unsqueeze(1)  # (batch_size, 1, hidden_size)
        energy = torch.bmm(v, energy)  # (batch_size, 1, seq_len)
        attention_weights = F.softmax(energy.squeeze(1), dim=1)  # (batch_size, seq_len)
        return attention_weights

class DecoderLSTM_Attention(nn.Module):
    def __init__(self, hidden_size, output_size, embedding_dim, num_layers=1):
        super(DecoderLSTM_Attention, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim + hidden_size, hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size * 2, output_size)
        self.attention = Attention(hidden_size)

    def forward(self, x, hidden, encoder_outputs):
        # x: (batch_size, 1)
        embedded = self.embedding(x)  # (batch_size, 1, embedding_dim)
        h_n, c_n = hidden
        # Use the last layer's hidden state
        decoder_hidden = h_n[-1]  # (batch_size, hidden_size)
        attention_weights = self.attention(decoder_hidden, encoder_outputs)  # (batch_size, seq_len)
        attention_weights = attention_weights.unsqueeze(1)  # (batch_size, 1, seq_len)
        context = torch.bmm(attention_weights, encoder_outputs)  # (batch_size, 1, hidden_size)
        lstm_input = torch.cat((embedded, context), dim=2)  # (batch_size, 1, embedding_dim + hidden_size)
        output, (h_n, c_n) = self.lstm(lstm_input, (h_n, c_n))
        output = torch.cat((output.squeeze(1), context.squeeze(1)), dim=1)  # (batch_size, hidden_size * 2)
        prediction = self.fc(output)  # (batch_size, output_size)
        return prediction, (h_n, c_n)

class Seq2SeqLSTM_Attention(nn.Module):
    def __init__(self, encoder, decoder, device='cuda'):
        super(Seq2SeqLSTM_Attention, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, src_lengths, trg_lengths, teacher_forcing_ratio=0.5):
        batch_size = src.size(0)
        max_trg_len = trg.size(1)
        trg_vocab_size = self.decoder.fc.out_features

        outputs = torch.zeros(batch_size, max_trg_len, trg_vocab_size).to(self.device)

        encoder_outputs, (hidden, cell) = self.encoder(src, src_lengths)

        input = trg[:, 0]  # <sos>
        hidden = (hidden, cell)  # Prepare hidden state for decoder

        for t in range(1, max_trg_len):
            output, hidden = self.decoder(input.unsqueeze(1), hidden, encoder_outputs)
            outputs[:, t] = output
            top1 = output.argmax(1)
            teacher_force = random.random() < teacher_forcing_ratio
            input = trg[:, t] if teacher_force else top1
        return outputs

# Instantiate the models with hyperparameter tuning
embedding_dim = 512
hidden_size = 1024
num_layers = 1
learning_rate = 0.0005
num_epochs = 5

# CUDA configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Instantiate the models with hyperparameter tuning
encoder = EncoderLSTM(eng_vocab_size, hidden_size, embedding_dim, num_layers).to(device)
decoder = DecoderLSTM_Attention(hidden_size, urd_vocab_size, embedding_dim, num_layers).to(device)
model = Seq2SeqLSTM_Attention(encoder, decoder, device=device).to(device)

# Training Setup
criterion = nn.CrossEntropyLoss(ignore_index=urd_word2idx['<pad>'])
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Modified training loop with model saving after each epoch
def train_model(model, iterator, optimizer, criterion, clip=1, teacher_forcing_ratio=0.5):
    model.train()
    epoch_loss = 0
    for i, (src, trg, src_lengths, trg_lengths) in enumerate(iterator):
        src, trg = src.to(device), trg.to(device)
        optimizer.zero_grad()
        output = model(src, trg, src_lengths, trg_lengths, teacher_forcing_ratio)
        output_dim = output.shape[-1]
        output = output[:, 1:].reshape(-1, output_dim)
        trg = trg[:, 1:].reshape(-1)
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(iterator)

def evaluate_model(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, (src, trg, src_lengths, trg_lengths) in enumerate(iterator):
            src, trg = src.to(device), trg.to(device)
            output = model(src, trg, src_lengths, trg_lengths, teacher_forcing_ratio=0)
            output_dim = output.shape[-1]
            output = output[:, 1:].reshape(-1, output_dim)
            trg = trg[:, 1:].reshape(-1)
            loss = criterion(output, trg)
            epoch_loss += loss.item()
    return epoch_loss / len(iterator)

# Training loop with saving model at each epoch
best_valid_loss = float('inf')
train_losses = []
valid_losses = []

for epoch in range(num_epochs):
    tf_ratio = max(0.5 - epoch * 0.02, 0)  # Gradually decrease teacher forcing ratio
    train_loss = train_model(model, train_loader, optimizer, criterion, teacher_forcing_ratio=tf_ratio)
    valid_loss = evaluate_model(model, val_loader, criterion)
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)

    # Save model after each epoch
    torch.save(model.state_dict(), f'seq2seq-lstm-attention-epoch{epoch+1}.pt')

    # Save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'seq2seq-lstm-attention-best.pt')

    print(f'Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.3f} | Val Loss: {valid_loss:.3f}')

# Load the best model
model.load_state_dict(torch.load('seq2seq-lstm-attention-best.pt'))

# Evaluation
def translate_sentence(model, sentence, eng_word2idx, urd_idx2word, max_length=50):
    model.eval()
    tokens = tokenize(normalize_string(sentence))
    indices = sentence_to_indices(tokens, eng_word2idx)
    src_tensor = torch.tensor(indices, dtype=torch.long).unsqueeze(0).to(device)
    src_length = [len(indices)]
    with torch.no_grad():
        encoder_outputs, (hidden, cell) = model.encoder(src_tensor, src_length)
    hidden = (hidden, cell)
    trg_indices = [urd_word2idx['<sos>']]
    for i in range(max_length):
        trg_tensor = torch.tensor([trg_indices[-1]], dtype=torch.long).to(device)
        with torch.no_grad():
            output, hidden = model.decoder(trg_tensor.unsqueeze(1), hidden, encoder_outputs)
            pred_token = output.argmax(1).item()
            trg_indices.append(pred_token)
            if pred_token == urd_word2idx['<eos>']:
                break
    trg_tokens = [urd_idx2word.get(idx, '<unk>') for idx in trg_indices]
    return ' '.join(trg_tokens[1:-1])  # Exclude <sos> and <eos>

def calculate_bleu(model, data, eng_word2idx, urd_idx2word):
    references = []
    hypotheses = []
    for idx in range(len(data)):
        src_tokens = data.iloc[idx]['English_tokens']
        trg_tokens = data.iloc[idx]['Urdu_tokens']
        src_sentence = ' '.join(src_tokens)
        reference = [' '.join(trg_tokens)]
        hypothesis = translate_sentence(model, src_sentence, eng_word2idx, urd_idx2word)
        references.append([reference[0].split()])
        hypotheses.append(hypothesis.split())
    bleu_score = corpus_bleu(references, hypotheses)
    return bleu_score

bleu_score = calculate_bleu(model, test_data, eng_word2idx, urd_idx2word)
print(f'Test BLEU Score (LSTM with Attention): {bleu_score*100:.2f}')

# Example Translations
test_sentences = [
    "How can I communicate with my parents?",
    "If you’ve asked yourself such questions, you’re not alone.",
    "Thumbs up",
    "Food was good but service was very slow."
]

for sentence in test_sentences:
    translation = translate_sentence(model, sentence, eng_word2idx, urd_idx2word)
    print(f'English: {sentence}')
    print(f'Urdu Translation (LSTM with Attention): {translation}')
    print()


Using device: cuda
Epoch 1/5 | Train Loss: 4.660 | Val Loss: 4.504
Epoch 2/5 | Train Loss: 3.294 | Val Loss: 4.144
Epoch 3/5 | Train Loss: 2.358 | Val Loss: 4.073
Epoch 4/5 | Train Loss: 1.751 | Val Loss: 4.132
Epoch 5/5 | Train Loss: 1.419 | Val Loss: 4.166


<ipython-input-40-568ad724f77c>:177: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('seq2seq-lstm-attention-best.pt'))


Test BLEU Score (LSTM with Attention): 18.64
English: How can I communicate with my parents?
Urdu Translation (LSTM with Attention): میں اپنے والدین سے کیسے کیسے کیسے کیسے ؟ ؟

English: If you’ve asked yourself such questions, you’re not alone.
Urdu Translation (LSTM with Attention): اگر آپ ایسا نہیں کہ تم نے آپ کو ایسا نہیں نہیں ہیں ۔

English: Thumbs up
Urdu Translation (LSTM with Attention): بہت خوب

English: Food was good but service was very slow.
Urdu Translation (LSTM with Attention): کھانا اچھا تھا لیکن سروس بہت سست تھی ۔

